# Create a Local Ethernet (Layer 2) Network

This notebook shows how to create an isolated local Ethernet and connect compute nodes to it.  


## Import the FABlib Library


In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     
fablib.show_config()

-----------------------------------  -----------------------------------------------
credmgr_host                         cm.fabric-testbed.net
orchestrator_host                    orchestrator.fabric-testbed.net
fabric_token                         /home/fabric/.tokens.json
project_id                           f8a6e0b0-ad14-47cb-9764-74c20ef3e4fc
bastion_username                     durbek_gafurov_0000000854
bastion_key_filename                 /home/fabric/work/fabric_config/bastionD
bastion_public_addr                  bastion-1.fabric-testbed.net
bastion_passphrase                   None
slice_public_key_file                /home/fabric/work/fabric_config/.ssh/id_rsa.pub
slice_private_key_file               /home/fabric/work/fabric_config/.ssh/id_rsa
fabric_slice_private_key_passphrase  None
fablib_log_file                      /tmp/fablib/fablib.log
fablib_log_level                     INFO
-----------------------------------  -----------------------------------------------


## (Optional): Query for Available Testbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [2]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
MASS         6  158/192  1356/1536  60134/60600    362/381                 2/2                             2/2                            10/10               2/2               3/3
SALT         6  176/192  1504/1536  60520/60600    375/381                 2/2                             2/2                            10/10               2/2               3/3
NCSA         6  152/192  1376/1536  60100/60600    372/381                 2/2                             2/2                            10/10               2/2               3/3
UTAH        10  242/320  2312/2560  115730/116400  620/635                 2/2

In [3]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
MASS         6  158/192  1356/1536  60134/60600    362/381                 2/2                             2/2                            10/10               2/2               3/3
SALT         6  176/192  1504/1536  60520/60600    375/381                 2/2                             2/2                            10/10               2/2               3/3
NCSA         6  152/192  1376/1536  60100/60600    372/381                 2/2                             2/2                            10/10               2/2               3/3
UTAH        10  242/320  2312/2560  115730/116400  620/635                 2/2

## Create the Experiment Slice

The following creates two nodes with basic NICs connected to an isolated local Ethernet.  

Two nodes are created and one NIC component is added to each node.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device. The VF is accessed by the node via PCI passthrough. Other NIC models are listed below. When using dedicated PCI devices the whole physical device is allocated to one node and the device is accessed by the node using PCI passthrough. Calling the `get_interfaces()` method on a component will return a list of interfaces. Many dedicated NIC components may have more than one port.  Either port can be connected to the network.

Next, add an `l2network` to the slice and pass the list of interfaces you want connected to this Ethernet. If all interfaces in the list are located on the same site, the network will automatically be a local Ethernet.  By default, a node is put on a random site.  If you want to ensure that your nodes are all on the same site you can specify the name of the site in the `add_node` methode.  You can use the `fablib.get_random_site()` method to get a random site name that can be used for both nodes.

NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

In [4]:
slice_name = 'MySlice1_1'

site = 'GPN' #fablib.get_random_site()
print(f"Site: {site}")

node1_name = 'Node1'
node2_name = 'Node2'
network_name='net1'
node1_nic_name = 'nic1'
node2_nic_name = 'nic2'
image='default_ubuntu_20'

cores = 2
ram = 8
disk = 10

Site: GPN


In [5]:
try:
    #Create Slice
    slice4 = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice4.add_node(name=node1_name, site=site, cores=cores, ram=ram, disk=disk)
    iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
    node1.set_image(image)
    
    # Node2
    node2 = slice4.add_node(name=node2_name, site=site, cores=cores, ram=ram, disk=disk)
    iface2 = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]
    node2.set_image(image)
    
    # Network
    net1 = slice4.add_l2network(name=network_name, interfaces=[iface1, iface2])

    #Submit Slice Request
    slice4.submit()
except Exception as e:
    print(f"Exception: {e}")


-----------  ------------------------------------
Slice Name   MySlice1_1
Slice ID     ab29dfe0-e43d-4d7f-add6-f4d10359bc3b
Slice State  StableOK
Lease End    2023-01-03 22:49:48 +0000
-----------  ------------------------------------

Retry: 14, Time: 168 sec

ID                                    Name    Site    Host                         Cores    RAM    Disk  Image              Management IP                          State    Error
------------------------------------  ------  ------  -------------------------  -------  -----  ------  -----------------  -------------------------------------  -------  -------
60f638fe-b9ba-46f7-8969-621ed3f24d50  Node1   GPN     gpn-w4.fabric-testbed.net        2      8      10  default_ubuntu_20  2610:e0:a04c:fab2:f816:3eff:fe32:1ce4  Active
22f4f39b-1a9b-49c1-aba4-d7a3fc084b05  Node2   GPN     gpn-w4.fabric-testbed.net        2      8      10  default_ubuntu_20  2610:e0:a04c:fab2:f816:3eff:fec4:cac7  Active

Time to stable 168 seconds
Running pos

## Observe the Slice's Attributes

### Print the slice 

In [6]:
try:
    slice4 = fablib.get_slice(name=slice_name)
    print(f"{slice4}")
except Exception as e:
    print(f"Exception: {e}")

-----------  ------------------------------------
Slice Name   MySlice1_1
Slice ID     ab29dfe0-e43d-4d7f-add6-f4d10359bc3b
Slice State  StableOK
Lease End    2023-01-03 22:49:48 +0000
-----------  ------------------------------------


## Print the Node List

In [7]:
try:
    slice4 = fablib.get_slice(name=slice_name)

    print(f"{slice4.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

ID                                    Name    Site    Host                         Cores    RAM    Disk  Image              Management IP                          State    Error
------------------------------------  ------  ------  -------------------------  -------  -----  ------  -----------------  -------------------------------------  -------  -------
60f638fe-b9ba-46f7-8969-621ed3f24d50  Node1   GPN     gpn-w4.fabric-testbed.net        2      8      10  default_ubuntu_20  2610:e0:a04c:fab2:f816:3eff:fe32:1ce4  Active
22f4f39b-1a9b-49c1-aba4-d7a3fc084b05  Node2   GPN     gpn-w4.fabric-testbed.net        2      8      10  default_ubuntu_20  2610:e0:a04c:fab2:f816:3eff:fec4:cac7  Active


## Print the Node Details

In [8]:
try:
    slice4 = fablib.get_slice(name=slice_name)
    for node in slice4.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  ---------------------------------------------------------------------------------------------------------------------------------------------------------
ID                 60f638fe-b9ba-46f7-8969-621ed3f24d50
Name               Node1
Cores              2
RAM                8
Disk               10
Image              default_ubuntu_20
Image Type         qcow2
Host               gpn-w4.fabric-testbed.net
Site               GPN
Management IP      2610:e0:a04c:fab2:f816:3eff:fe32:1ce4
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/.ssh/id_rsa -J durbek_gafurov_0000000854@bastion-1.fabric-testbed.net ubuntu@2610:e0:a04c:fab2:f816:3eff:fe32:1ce4
-----------------  ---------------------------------------------------------------------------------------------------------------------------------------------------------
-----------------  ------------------------------------------------------------------------------------------

## Print the Interfaces

In [9]:
try:
    slice4 = fablib.get_slice(name=slice_name)
    
    print(f"{slice4.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

Name           Node    Network      Bandwidth  VLAN    MAC                Physical OS Interface    OS Interface
-------------  ------  ---------  -----------  ------  -----------------  -----------------------  --------------
Node1-nic1-p1  Node1   net1                 0          16:54:ED:0A:AA:3D  ens7                     ens7
Node2-nic2-p1  Node2   net1                 0          16:9D:DE:C6:0E:E0  ens7                     ens7


##  Configure IP Addresses

Some experiments use FABRIC layer 2 networks to enable deploying non-IP layer 3 networks.  If this describes your experiment, your nodes and network are ready. You can now login to the nodes and deploy your experiment.

Most users will want to configure IP addresses on their new nodes.  FABlib provides some useful methods to help you configure basic IP addresses. 

### Pick a Subnet

Create a subnet and list of available IP addresses. All objects are Python IP management objects. You can use either IPv4 or IPv6 subnets and addresses.

In [10]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet = IPv4Network("192.168.1.0/24")
    available_ips = list(subnet)[1:]
except Exception as e:
    print(f"Exception: {e}")

### Configure Node1

Get the node and the interface you wish to configure.  You can use `node.get_interface` to get the interface that is connected to the specified network.  Then `pop` an IP address from the list of available IPs and call `iface.ip_addr_add` to set the IP and subnet.  

Optionally, use the `node.execute()` method to show the results of adding the IP address.

In [11]:
try:
    node1 = slice4.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network_name) 
    node1_addr = available_ips.pop(0)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 16:54:ed:0a:aa:3d brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.1/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::1454:edff:fe0a:aa3d/64 scope link 
       valid_lft forever preferred_lft forever



### Configure Node2

Repeat the steps to add the next available IP to the second node.

In [12]:
try:
    node2 = slice4.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network_name)  
    node2_addr = available_ips.pop(0)
    node2_iface.ip_addr_add(addr=node2_addr, subnet=subnet)
    
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 16:9d:de:c6:0e:e0 brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.2/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::149d:deff:fec6:ee0/64 scope link 
       valid_lft forever preferred_lft forever



## Run the Experiment

We will find the ping round trip time for this pair of sites.


In [13]:
try:
#     node1 = slice4.get_node(name=node1_name)        

    stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

PING 192.168.1.2 (192.168.1.2) 56(84) bytes of data.
64 bytes from 192.168.1.2: icmp_seq=1 ttl=64 time=0.340 ms
64 bytes from 192.168.1.2: icmp_seq=2 ttl=64 time=0.069 ms
64 bytes from 192.168.1.2: icmp_seq=3 ttl=64 time=0.073 ms
64 bytes from 192.168.1.2: icmp_seq=4 ttl=64 time=0.087 ms
64 bytes from 192.168.1.2: icmp_seq=5 ttl=64 time=0.101 ms

--- 192.168.1.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4077ms
rtt min/avg/max/mdev = 0.069/0.134/0.340/0.103 ms




---

## Installing iperf

In [14]:
stdout, stderr = node1.execute(f'sudo apt update && sudo apt install -y iperf iperf3')
print (stdout)
print (stderr)

stdout, stderr = node2.execute(f'sudo apt update && sudo apt install -y iperf iperf3')
print (stdout)
print (stderr)

Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [1895 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [311 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 c-n-f Metadata [11.5 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1385 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted Translation-en [195 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 c-n-f Metadata [596 B]
Get:12 http://security.ubuntu.com/ubuntu focal-security/univ

## Setting Parameters

In [15]:
stdout, stderr = node1.execute('echo "net.core.rmem_max = 2147483647\nnet.core.wmem_max = 2147483647\nnet.ipv4.tcp_rmem = 4096 87380 2147483647\nnet.ipv4.tcp_wmem = 4096 65536 2147483647\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')
print (stdout)
print (stderr)

net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control=htcp
net.ipv4.tcp_mtu_probing=1
net.core.default_qdisc = fq

net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control = htcp
net.ipv4.tcp_mtu_probing = 1
net.core.default_qdisc = fq




In [16]:
stdout, stderr = node2.execute('echo "net.core.rmem_max = 2147483647\nnet.core.wmem_max = 2147483647\nnet.ipv4.tcp_rmem = 4096 87380 2147483647\nnet.ipv4.tcp_wmem = 4096 65536 2147483647\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')
print (stdout)
print (stderr)

net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control=htcp
net.ipv4.tcp_mtu_probing=1
net.core.default_qdisc = fq

net.core.rmem_max = 2147483647
net.core.wmem_max = 2147483647
net.ipv4.tcp_rmem = 4096 87380 2147483647
net.ipv4.tcp_wmem = 4096 65536 2147483647
net.ipv4.tcp_congestion_control = htcp
net.ipv4.tcp_mtu_probing = 1
net.core.default_qdisc = fq




In [17]:
#server
stdout, stderr = node1.execute('tmux new -d \'iperf -s -f K\'') 
print (stdout)
print (stderr)

In [30]:
#client
stdout, stderr = node2.execute(f'iperf -c 192.168.1.1 -M  -P 50 -w 999M') #-f g
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 192.168.1.1, TCP port 5001
TCP window size: 1.95 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[  3] local 192.168.1.2 port 36996 connected with 192.168.1.1 port 5001
[ ID] Interval       Transfer     Bandwidth
[  3]  0.0-10.0 sec  22.9 GBytes  19.7 Gbits/sec

iperf: ignoring extra argument -- 50



In [20]:
#client
stdout, stderr = node2.execute(f'iperf -c 192.168.1.1 -P 50 -w 999M') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 192.168.1.1, TCP port 5001
TCP window size: 1.95 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 10] local 192.168.1.2 port 59322 connected with 192.168.1.1 port 5001
[ 29] local 192.168.1.2 port 59360 connected with 192.168.1.1 port 5001
[  3] local 192.168.1.2 port 59308 connected with 192.168.1.1 port 5001
[ 13] local 192.168.1.2 port 59328 connected with 192.168.1.1 port 5001
[  9] local 192.168.1.2 port 59318 connected with 192.168.1.1 port 5001
[  8] local 192.168.1.2 port 59320 connected with 192.168.1.1 port 5001
[ 15] local 192.168.1.2 port 59332 connected with 192.168.1.1 port 5001
[  4] local 192.168.1.2 port 59310 connected with 192.168.1.1 port 5001
[ 17] local 192.168.1.2 port 59336 connected with 192.168.1.1 port 5001
[  6] local 192.168.1.2 port 59314 connected with 192.168.1.1 port 5001
[ 30] local 192.168.1.2 port 59362 connected with 

In [33]:
#client
stdout, stderr = node2.execute(f'iperf -c 192.168.1.1 -M 2000 -w 9999999M') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 192.168.1.1, TCP port 5001
TCP window size: 2.00 GByte (WARNING: requested 1.62 GByte)
------------------------------------------------------------
[  3] local 192.168.1.2 port 59314 connected with 192.168.1.1 port 5001
[ ID] Interval       Transfer     Bandwidth
[  3]  0.0-10.0 sec  18.4 GBytes  15.8 Gbits/sec




## Delete the Slice

Please delete your slice when you are done with your experiment.

In [19]:
# try:
#     slice = fablib.get_slice(name=slice_name)
#     slice.delete()
# except Exception as e:
#     print(f"Exception: {e}")